# Lecture 1:  Data transformation

<div style="border: 1px double black; padding: 10px; margin: 10px">

**Goals for today's lecture:**
* Learn [how to manipulate data](#Data-manipulation), including:
    * [Filtering data](#Row-operation-#1:-Filtering-data)
    * [Arranging (sorting) rows](#Row-operation-#2:-Sorting)
    * [Finding distinct rows](#Row-operation-#3:-distinct())
    * [Selecting subsets of columns](#Column-operation-#1:-select()
    
This lecture note corresponds to [Chapter 4](https://r4ds.hadley.nz/data-transform.html) of your book, and we will also use some ideas from Chapters [14](https://r4ds.hadley.nz/logicals.html)-[15](https://r4ds.hadley.nz/numbers.html).
</div>


## Data manipulation
Manipulating data is an important part of data science--perhaps the most important! As a data scientist, most of your time will be spent simply getting your data into a format that you can analyze:
![data manipulation plot](https://www.datanami.com/wp-content/uploads/2020/07/Anaconda_1.png)

https://www.datanami.com/2020/07/06/data-prep-still-dominates-data-scientists-time-survey-finds/

There are a lot of built-in commands for data manipulation in R:
```{r}
# Traditional data manipulation commands in R
subset()
aggregate()
merge()
reshape()
```
These commands are old and somewhat difficult to use. Instead of the traditional commands, we are going to focus on the `dplyr` package for filtering data. They provide a nice suite of replacements for the traditional commands, which have a consistent, unified interface and interoperate nicely with each other.

The `dplyr` package is part of `tidyverse`, so it is already loaded once we run `library(tidyverse)` (which we remember from last lecture that we should do each time we start R).

We will be using the `nycflights13` data set for this lecture. It does not come with tidyverse. If you are running Jupyter on your own computer you will first need to `install.packages("nycflights13")`.  This data set is about flights departing from the NYC area in 2013.  

In [1]:
install.packages('nycflights13')
library(tidyverse)
library(nycflights13)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
print(flights)

# A tibble: 336,776 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# … with 336,766 more rows,

Tibble is similar to dataframe and we will learn more about it later in the course.   For now, you can interpret it as a dataframe.

Notice the types of the variables in `flights`. They include:

* **int** integers
* **dbl** double precision floating point numbers
* **chr** character vectors, or strings
* **dttm** date-time (a date along with a time)

Other types available in R but not represented above include:

* **lgl** logical (either `TRUE` or `FALSE`)
* **fctr** factor (categorical variable with a fixed number of possible values)
* **date** date

## What's a data frame?

Our main goal in R is to work with data, and one of the most fundamental objects in R is the *data frame*. Think of a data frame as a container for a bunch of *vectors* of data:

![dataframe](https://garrettgman.github.io/images/tidy-2.png)

## Row operation #1: Filtering data
The first operation we'll learn about is filtering. Filtering is interpereted to mean "keep only the rows whose columns match these criteria". The syntax for the `filter` command is
```{r}
filter(<TABLE>, <LOGICAL CRITERIA>)
```
This commands returns a new tibble whose rows all match the specified criteria.

### Types of logical criteria
For those who are new to programming, we now briefly review [boolean logic](https://en.wikipedia.org/wiki/Boolean_algebra). The basic logical operators in R are `>`, `>=`, `<`, `<=`, `!=` (not equal), and `==` (equal). The first four are used for comparing numbers and function exactly as in mathematics:
```{r}
> 1 > 1
[1] FALSE
> 1 >= 1
[1] TRUE
> 2.5 < 3
[1] TRUE
> 2.5 <= 3
[1] TRUE
```

### Assignment vs. equality
An extremely common mistake for beginner programmers is to confuse `=` and `==` ("double equals") when writing code. As we have seen,
- `=` is used for
    - assigning a value to a variable, and
    - passing a named parameter into a function.
- `==` is used for testing equality.


```{r}
> a = 1  # assigns the integer 1 to a
> b = 2  # assigns the integer 2 to b
> a == 1 # tests that a equals 1
[1] TRUE
> b == 1 # tests that b equals 1
[1] FALSE
```

### Boolean operations
Logical expressions are combined using *boolean operations*. The basic boolean operations are `and`, `or`, and `not`, denoted `&`, `|` and `!` respectively.

In [ ]:
! FALSE
FALSE | TRUE
! TRUE

[1] TRUE

[1] TRUE

[1] FALSE

There are also doubled versions of `&` and `|` denoted `&&` and `||`. Do not use these yet. We will return to them later in the course when we discuss programming and control flow.

R abbreviates `TRUE` and `FALSE` as `T` and `F`:

In [2]:
c(T, F)

[1]  TRUE FALSE

### Vectorization
What happens when we ask whether a *vector* satisfies a logical condition? R returns a new logical vector with the same number of entries, showing whether each entry satisfies that condition:

In [3]:
# Examples of logical conditions
a = c(1:10)
print(a)
a > 3

 [1]  1  2  3  4  5  6  7  8  9 10


[1] FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE

### Testing for membership
Another useful operator is `%in%`:
```r
x %in% y
```

return `TRUE` if the value `x` is found in the vector `y`:

In [ ]:
"a" %in% c(1, 2, 3)
("a" == 1) | ("a" == 2) | ("a" == 3)

[1] FALSE

[1] FALSE

## Missing data
Something you will often encounter when working with real data are missing observations. R has a special value, `NA` , for representing missing data. You can think of the value of `NA` as "I don't know". Thus, logical and mathematical operations involving `NA` will again return `NA`, so that `NA`s "propagate through" the computation:

In [ ]:
NA + 5 # 5 + I-don't-know = I-don't-know

[1] NA

In [ ]:
1 < NA  # Is 1 less that I-don't-know? I don't know.

[1] NA

In [ ]:
NA < NA  # Is I-don't-know greater than I-don't-know? I don't know.
mean(c(1, NA, 3), na.rm=F) # And so forth.

[1] NA

[1] NA

In [ ]:
NA == 1

[1] NA

Since you cannot test `NA`s for equality, R has a special function for determining whether a value is `NA`:

In [ ]:
5 == NA

is.na(NA)
is.na(1)

[1] NA

[1] TRUE

[1] FALSE

In [ ]:
# Examples of missing values

### Examples of filtering
Let's use what we have just learned to evaluate some simple queries on the `flights` dataset. Let's first narrow down to all flights that departed from JFK

In [ ]:
# Filter to all flights that departed JFK
filter(flights, origin == 'JFK')

How does all this work? Basically what R does here is create a logical vector that has one entry for each **row** of the input data frame. Then, it returns a new data frame which contains all the rows where the logical vector evaluated to `True`.

## Remark:
Using `==` for testing equality is very important in `R`.  `R` will yield an error if you use `=`.  

In [ ]:
# what happens when you forget to use == in filter?
filter(flights, origin = 'JFK')

### Counting matches
Sometimes we just want to know how many observations match a given filter. The `nrow()` command can be used to count the number of rows in a data table.

In [6]:
# example of using nrow
nrow(flights)

[1] 336776

## 🤔 Quiz
How many flights in the table departed from JFK?

<ol style="list-style-type: upper-alpha;">
    <li>Fewer than 50,000</li>
    <li>Between 50,000 and 80,000</li>
    <li>Between 80,000 and 90,000</li>
    <li>Between 90,000 and 100,000</li>
    <li>More than 100,000</li>
</ol>

In [ ]:
# Code to get number of flights that departed from JFK


Let us try to calculate how many flights with missing departure time in our data.

In [ ]:
# How many flights have a missing departure time
filter(flights, is.na(dep_time))

In [14]:
flights$month %>% unique

[1]  1 10 11 12  2  3  4  5  6  7  8  9

How about the number of flights departing between Jan and Mar?

In [ ]:
# no. of flights departing between Jan and Mar
filter(flights, month %in% c(...))

### Row operation #1.5: `head()`/`tail()`
A particularly useful kind of `filter()` is to only keep the first, or last, rows. This happens so often in practical data science, that there are built-in commands to do it:

In [20]:
# first 6 rows of flights
head(...)

ERROR: Error in eval(expr, envir, enclos): '...' used in an incorrect context


In [ ]:
# last 6 rows of flights
tail(...)

## Row operation #2: Sorting
Often we want to sort our data based on one or more column values. This can arise for several reasons:
- The data have some natural order (for example, chronological)
- We want to learn about "extreme" features of the data:
  - "What was the most delayed flight?"
  - "What was the hottest day of the year?"
  - "Who was the tallest NBA player in 2010?"

`arrange()` changes the order of the rows based on the value of the columns.
- It takes a data frame and a set of column names (or more complicated expressions) to order by.
- If you provide multiple expressions to order by, it uses the second one to break ties in the first one, third one to break ties in the second one, and so on.
- By default, things are sorted in **ascending** order.

In [ ]:
# sort flights in ascending order by month and day
arrange(flights, month, day)

We sorted the data by month and day, so the top-most rows have the earliest month, folllowed by day.

If you want to sort in **descending** order, write `desc(<column>)` instead:

In [ ]:
# flights in descending order of month

## 🤔 Quiz
What's the maximum value of the dep_delay found in this dataset?

<ol style="list-style-type: upper-alpha;">
    <li>1301 minutes</li>
    <li>1137 minutes</li>
    <li>1242 minutes</li>
    <li>4 minutes</li>
    <li>49 minutes</li>
</ol>

## Row operation #3: `distinct()`
`distinct()` finds all the unique rows in a dataset. If you supply column names, it will find all unique combinations of those columns.

Obviously there are many flights scheduled for each day. But is there a flight scheduled for *every* day?

In [ ]:
# All the unique days in flights
distinct(flights, year, month, day)

## 🤔 Quiz
How many unique *airplanes* are represented in the `flights` table?

<ol style="list-style-type: upper-alpha;">
    <li>3,030</li>
    <li>4,043</li>
    <li>4,044</li>
    <li>90,210</li>
    <li>Can't be determined from the table</li>
</ol>

(Hint: every airplane has a unique tail number.)

In [26]:
flights %>% print

# A tibble: 336,776 × 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# ℹ 336,766 more rows
# ℹ 1

In [28]:
# analysis of unique airplanes
nrow(distinct(flights, tailnum))

[1] 4044

Another way of finding the same result - we will learn this notation later.

In [31]:
length(flights$tailnum %>% unique)

[1] 4044

## Column operation #1: `select()`
`select()` can help you to narrow down a large dataset by just focusing on the variables you’re interested in. `select()` allows you to rapidly zoom in on a useful subset using operations based on the names of the variables., but you can still get the general idea of how it works:

In [32]:
tbl <- select(flights, year, month, day, `departure time` = dep_time, arrival_time = arr_time)
head(tbl)

year,month,day,departure time,arrival_time
<int>,<int>,<int>,<int>,<int>
2013,1,1,517,830
2013,1,1,533,850
2013,1,1,542,923
2013,1,1,544,1004
2013,1,1,554,812
2013,1,1,554,740


Note that `select` drops any variables not explicitly mentioned. To just rename some variables while keeping all others, use `rename`.

In [ ]:
head(rename(flights, departure_time = dep_time, arrival_time = arr_time))

year,month,day,departure_time,sched_dep_time,dep_delay,arrival_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00


If there are a lot of variables, you can save yourself some typing by using `:` and `-` in combination with select. The colon operator selects a range of variables:

In [ ]:
head(select(flights, year:day))

year,month,day
<int>,<int>,<int>
2013,1,1
2013,1,1
2013,1,1
2013,1,1
2013,1,1
2013,1,1


The negative sign lets you select everything but certain columns:

In [ ]:
# select everything except the day and year columns
select(flights, -day, ...)

If you want to bring a few variables at the beginning, you can use `everything()` to refer to the remaining variables.

In [ ]:
# bring dep_time, arr_time, day, month, year to front
select(..., everything())

In addition, there are some helper functions that only work inside `select()`.

* `starts_with()`, `ends_with()`, `contains()`
* `matches()`
* `num_range()`

You can consult the documentation or type `?select` at the prompt to learn more about these. Here's just one example of their use.

In [40]:
# select all the variables that contain the word "time"
select(flights, contains(...))

dep_time,sched_dep_time,arr_time,sched_arr_time,air_time,time_hour
<int>,<int>,<int>,<int>,<dbl>,<dttm>
517,515,830,819,227,2013-01-01 05:00:00
533,529,850,830,227,2013-01-01 05:00:00
542,540,923,850,160,2013-01-01 05:00:00
544,545,1004,1022,183,2013-01-01 05:00:00
554,600,812,837,116,2013-01-01 06:00:00
554,558,740,728,150,2013-01-01 05:00:00
555,600,913,854,158,2013-01-01 06:00:00
557,600,709,723,53,2013-01-01 06:00:00
557,600,838,846,140,2013-01-01 06:00:00
